# Importing data

In [9]:
import openeo
import json
import folium
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# connect to the federated backend
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [11]:
connection.list_collection_ids()
connection.describe_collection("SENTINEL_5P_L2")

{'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['CO',
    'HCHO',
    'NO2',
    'O3',
    'SO2',
    'CH4',
    'AER_AI_340_380',
    'AER_AI_354_388',
    'CLOUD_BASE_PRESSURE',
    'CLOUD_TOP_PRESSURE',
    'CLOUD_BASE_HEIGHT',
    'CLOUD_TOP_HEIGHT',
    'CLOUD_OPTICAL_THICKNESS',
    'CLOUD_FRACTION',
    'dataMask']},
  't': {'extent': ['2018-04-30T00:18:50Z', None],
   'step': 'P1D',
   'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': 4326,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-85, 85],
   'reference_system': 4326,
   'type': 'spatial'}},
 'description': 'Sentinel 5 Precursor imagery captured by TROPOMI sensor.\n\nThis dataset only supports loading one band at a time.',
 'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
  'temporal': {'interval': [['2018-04-30T00:18:50Z', None]]}},
 'id': 'SENTINEL_5P_L2',
 'license': 'proprietary',
 'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
   'rel': 'root',
   'type': 'application/json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2',
   'rel': 'self',
   'type': 'application/json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
   'rel': 'parent',
   'type': 'application/json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/queryables',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
   'type': 'application/schema+json'},
  {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/items',
   'rel': 'items',
   'type': 'application/geo+json'}],
 'providers': [],
 'sci:citation': 'Modified Copernicus Sentinel data [Year]/Sentinel Hub',
 'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
  'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
  'https://docs.sentinel-hub.com/api/latest/stac/s5p/v1.0.0/schema.json'],
 'stac_version': '1.0.0',
 'summaries': {'eo:bands': [{'data_type': 'float32',
    'description': 'Carbon monoxide',
    'name': 'CO',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.1], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Formaldehyde',
    'name': 'HCHO',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.001], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Nitrogen oxide',
    'name': 'NO2',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.0003], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Ozone',
    'name': 'O3',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.36], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Sulphur dioxide',
    'name': 'SO2',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [0.01], 'minimum': [0]}},
   {'data_type': 'float32',
    'description': 'Methane',
    'name': 'CH4',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [2000], 'minimum': [1600]}},
   {'data_type': 'float32',
    'description': 'UV (Ultraviolet) Aerosol Index calculated based on wavelengths of 340 nm and 380 nm',
    'name': 'AER_AI_340_380',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
    'statistics': {'maximum': [5], 'minimum': [-1]}},
   {'data_type': 'float32',
    'description': 'UV (Ultraviolet) Aerosol Index calculated based on wavelengths of 354 nm and 388 nm',
    'name': 'AER_AI_354_388',
    'openeo:gsd': {'unit': '°',
     'value': [0.054563492063483, 0.034722222222216]},
   

In [ ]:
import openeo
import json
import xarray as xr

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

def read_json(filename: str) -> dict:
    with open(filename) as input:
        field = json.load(input)
    return field

collection = 'SENTINEL2_L2A'
dates = ["2021-08-01", "2021-08-08"]
aoi = read_json("data/germany.geojson")
bands = ['B03', 'B04', 'B08', 'B8A', 'B11']

cube = connection.load_collection(
    collection,
    temporal_extent=dates,
    spatial_extent=aoi,
    bands=bands,
    max_cloud_cover=50

)

cube = cube.aggregate_temporal_period(reducer="mean", period="week")

# job configuration
job_options = {
    "executor-memory": "2G",
    "executor-memoryOverhead": "3G",
    "python-memory": "3G",
}
# running a batch job

nom_fichier = ""

cube.execute_batch(
    outputfile=nom_fichier,
    job_options=job_options,
)

ds = xr.open_dataset(nom_fichier)
ds

In [ ]:
import openeo
import json
import xarray as xr

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

def read_json(filename: str) -> dict:
    with open(filename) as input:
        field = json.load(input)
    return field

collection = 'SENTINEL3_SLSTR_L2_LST'
dates = ["2021-08-01", "2021-09-01"]
aoi = read_json("data/germany.geojson")
bands = ['LST']

cube = connection.load_collection(
    collection,
    temporal_extent=dates,
    spatial_extent=aoi,
    bands=bands,
    max_cloud_cover=50

)

#red = cube.band("B04")
#nir = cube.band("B08")
#cube = (nir - red) / (nir + red)

cube = cube.aggregate_temporal_period(reducer="mean", period="month")

# job configuration
job_options = {
    "executor-memory": "2G",
    "executor-memoryOverhead": "3G",
    "python-memory": "3G",
}
# running a batch job

nom_fichier = "data/bands.nc"

cube.execute_batch(
    outputfile=nom_fichier,
    job_options=job_options,
)

ds = xr.open_dataset(nom_fichier)
ds